In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import io
from tqdm import tqdm


# data load

In [3]:
pitcher = pd.read_csv('/Users/sjung/Desktop/Data_Science_Lab/EDA_project/datasets/pitcherRecords.csv')
hitter = pd.read_csv('/Users/sjung/Desktop/Data_Science_Lab/EDA_project/datasets/hitterRecords.csv')
playerInfo = pd.read_excel('/Users/sjung/Desktop/Data_Science_Lab/EDA_project/datasets/playerInfo_v.01.xlsx')

In [4]:
playerInfo.head()

,선수명,id,등번호,생년월일,포지션,구체적포지션,신장(cm),체중 (kg),팀명,경력,입단 계약금(만원),연봉(만원),입단년도
0,차영화,10005,82.0,1957-06-27,NaN,우투우타,178.0,82.0,해태,서림초-전남중-광주제일고-동신전문대-해태,NaN,NaN,82해태
1,황태환,10082,NaN,1952-08-06,투수,좌투좌타,NaN,NaN,OB,한양대,NaN,NaN,NaN
2,황기선,10182,NaN,1958-06-02,투수,우투우타,NaN,NaN,해태,유한공고,NaN,NaN,NaN
3,조창수,20002,NaN,1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,김광현,20003,29.0,1988-07-22,투수,좌투좌타,188.0,88.0,NaN,NaN,NaN,NaN,NaN


# 투수와 타자로 나누기

In [8]:
pitcherInfo = playerInfo[playerInfo['선수명'].isin(pitcher['이름']) &\
                        playerInfo['id'].isin(pitcher['id'])].reset_index(drop = True)
pitcherInfo

,선수명,id,등번호,생년월일,포지션,구체적포지션,신장(cm),체중 (kg),팀명,경력,입단 계약금(만원),연봉(만원),입단년도
0,황태환,10082,NaN,1952-08-06,투수,좌투좌타,NaN,NaN,OB,한양대,NaN,NaN,NaN
1,황기선,10182,NaN,1958-06-02,투수,우투우타,NaN,NaN,해태,유한공고,NaN,NaN,NaN
2,소형준,50030,30.0,2001-09-16,투수,우투우타,189.0,92.0,KT,호암초(의정부리틀)-구리인창중-유신고,36000.0,32000.0,20KT
3,이강준,50036,2.0,2001-12-14,투수,우투우타,180.0,80.0,KT,서당초-설악중-설악고-KT-롯데-키움-상무,9000.0,3200.0,20KT
4,데스파이네,50040,40.0,1987-04-04,투수,우투우타,182.0,92.0,KT,미국 Hermanos Martinez Tamayo(고),40125.0,93625.0,20KT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581,박정진,99737,75.0,1976-05-27,NaN,좌투좌타,183.0,88.0,한화,청주중앙초-청주중-세광고-연세대-한화,NaN,NaN,99한화
1582,오창선,99738,27.0,1976-06-02,투수,우투우타,183.0,83.0,한화,대전고-홍익대,NaN,NaN,NaN
1583,김병준,99742,11.0,1977-04-01,투수,우투우타,184.0,84.0,한화,대전고-고려대,NaN,NaN,NaN
1584,전하진,99767,67.0,1979-02-11,투수,우투우타,180.0,78.0,한화,세광고-제주관광대,NaN,NaN,NaN


In [9]:
hitterInfo = playerInfo[playerInfo['선수명'].isin(hitter['이름']) &\
                        playerInfo['id'].isin(hitter['id'])].reset_index(drop = True)
hitterInfo

,선수명,id,등번호,생년월일,포지션,구체적포지션,신장(cm),체중 (kg),팀명,경력,입단 계약금(만원),연봉(만원),입단년도
0,차영화,10005,82.0,1957-06-27,NaN,우투우타,178.0,82.0,해태,서림초-전남중-광주제일고-동신전문대-해태,NaN,NaN,82해태
1,우용득,20001,89.0,1950-02-13,NaN,우투우타,172.0,83.0,삼성,대구상고-삼성,NaN,NaN,NaN
2,손상대,30003,79.0,1954-10-20,NaN,우투우타,175.0,79.0,삼성,신암초-경북중-경북고-한양대-삼성,NaN,NaN,82삼성
3,유지훤,30004,74.0,1955-10-04,NaN,우투우타,174.0,72.0,OB,대신초-대동중-대광고-OB,NaN,NaN,82OB
4,김인식,40006,70.0,1953-11-09,NaN,우투우타,168.0,80.0,MBC,동대문상고-성균관대,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,데이비스,99725,25.0,1969-10-03,외야수,좌투좌타,183.0,84.0,한화,미국 시카고ORR고,NaN,NaN,NaN
2445,황우구,99731,14.0,1976-05-03,내야수,우투우타,183.0,80.0,한화,동산고-인하대,NaN,NaN,NaN
2446,박정진,99737,75.0,1976-05-27,NaN,좌투좌타,183.0,88.0,한화,청주중앙초-청주중-세광고-연세대-한화,NaN,NaN,99한화
2447,김병준,99742,11.0,1977-04-01,투수,우투우타,184.0,84.0,한화,대전고-고려대,NaN,NaN,NaN


# Data preprocessing
statiz의 링크가 http://www.statiz.co.kr/player.php?opt=1&name=윤석민&birth=1986-07-24  
이런 형태로 저장돼 있기 때문에, 생년월일 column을 '1986-07-24'의 형식으로 바꿔줘야 한다.

In [10]:
hitter_simple = pd.DataFrame(columns = ['name', 'id', 'birthday'])
pitcher_simple = pd.DataFrame(columns = ['name', 'id', 'birthday'])

In [13]:
hitter_simple['birthday'] = pd.to_datetime(hitterInfo['생년월일'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
hitter_simple['name'] = hitterInfo['선수명']
hitter_simple['id'] = hitterInfo['id']
hitter_simple

,name,id,birthday
0,차영화,10005,1957-06-27
1,우용득,20001,1950-02-13
2,손상대,30003,1954-10-20
3,유지훤,30004,1955-10-04
4,김인식,40006,1953-11-09
...,...,...,...
2444,데이비스,99725,1969-10-03
2445,황우구,99731,1976-05-03
2446,박정진,99737,1976-05-27
2447,김병준,99742,1977-04-01


In [14]:
pitcher_simple['birthday'] = pd.to_datetime(pitcherInfo['생년월일'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
pitcher_simple['name'] = pitcherInfo['선수명']
pitcher_simple['id'] = pitcherInfo['id']
pitcher_simple

,name,id,birthday
0,황태환,10082,1952-08-06
1,황기선,10182,1958-06-02
2,소형준,50030,2001-09-16
3,이강준,50036,2001-12-14
4,데스파이네,50040,1987-04-04
...,...,...,...
1581,박정진,99737,1976-05-27
1582,오창선,99738,1976-06-02
1583,김병준,99742,1977-04-01
1584,전하진,99767,1979-02-11


# Crawling

## Pitcher data

In [15]:
def extract_pitcher_data(url, name, id):
    response = requests.get(url, verify = False)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.select("body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(4) > div > div > table")

    colhead_rows = soup.find_all('tr', class_='colhead_stz0')
    for tr in colhead_rows[2:]:
        tr.decompose()
    
    if len(table) == 0: # can't find the player data
        return pd.DataFrame()
    else:
        pitcher_df = pd.read_html(io.StringIO(str(table)))[0][:-1] # Remove "통산" row
        pitcher_df.columns = pitcher_df.columns.droplevel(0)
        
        # insert player name, id, birthday
        extracted_pitcher_info = pitcher_simple[(pitcher_simple['name'] == name) & (pitcher_simple['id'] == id)]
        extracted_pitcher_info = extracted_pitcher_info[extracted_pitcher_info.columns[::-1]]
        
        for column in extracted_pitcher_info:
            pitcher_df.insert(0, column, extracted_pitcher_info[column].values[0])
    
        return pitcher_df


In [16]:
pitcher_columns = ['name', 'id', 'birthday', '연도', '팀', '나이', '출장', '완투', '완봉', '선발',
       '승', '패', '세', '홀드', '이닝', '실점', '자책', '타자', '안타', '2타', '3타',
       '홈런', '볼넷', '고4', '사구', '삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP',
       'ERA+', 'FIP+', 'WAR', 'WPA']

### Extract

In [17]:
pitcher_url_head = 'http://www.statiz.co.kr/player.php?opt=1&name='
pitcher_url_middle = '&birth='
pitcher_url_tail = '&re=1'

In [23]:
pitcher_data_list = list()
for i in tqdm(range(len(pitcher_simple))):
    pitcher_name = pitcher_simple.iloc[i]['name']
    pitcher_birthday = pitcher_simple.iloc[i]['birthday']
    pitcher_id = pitcher_simple.iloc[i]['id']
    pitcher_url = pitcher_url_head+pitcher_name+pitcher_url_middle+pitcher_birthday+pitcher_url_tail
    pitcher_data = extract_pitcher_data(pitcher_url, pitcher_name, pitcher_id)
    pitcher_data_list = pitcher_data_list + pitcher_data.values.tolist()
    

100%|██████████| 1586/1586 [04:52<00:00,  5.41it/s]


In [24]:
pitcher_data_df = pd.DataFrame(pitcher_data_list, columns = pitcher_columns)


### NaN처리
한 시즌 중에 선수가 팀을 옮긴 경우, 총 세가지의 column이 생기게 됨.  
ex.  
|      |   name |    id |   birthday |   연도 |      팀 | 나이 | 출장 |
|------|-------:|------:|-----------:|-------:|--------:|-----:|-----:|
| 7064 | 송신영 | 99314 | 1977-03-01 | 2011.0 | 넥센,LG |   34 |   62 |
| 7065 | 송신영 | 99314 | 1977-03-01 |    NaN |    넥센 |   34 |   43 |
| 7066 | 송신영 | 99314 | 1977-03-01 |    NaN |      LG |   34 |   19 |  

이렇게 연도가 NaN으로 표시되는 문제 해결

In [31]:
pitcher_data_df_preprocessed = pitcher_data_df.copy(deep=True)
pitcher_data_df_preprocessed['연도'] = pitcher_data_df['연도'].ffill()

In [32]:
pitcher_data_df_preprocessed

,name,id,birthday,연도,팀,나이,출장,완투,완봉,선발,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
0,황태환,10082,1952-08-06,1982.0,OB,30,27,2,0,6,...,42,0,4,3.86,4.85,1.31,100.9,80.2,1.08,NaN
1,황태환,10082,1952-08-06,1983.0,OB,31,39,0,0,2,...,60,0,0,2.65,3.50,1.38,123.6,93.5,2.02,NaN
2,황태환,10082,1952-08-06,1984.0,OB,32,17,0,0,3,...,19,0,0,2.48,3.44,1.28,135.7,97.6,0.92,NaN
3,황태환,10082,1952-08-06,1985.0,OB,33,35,1,1,6,...,50,0,2,3.53,3.23,1.23,101.1,110.5,2.10,NaN
4,황태환,10082,1952-08-06,1986.0,OB,34,28,0,0,1,...,36,0,1,2.45,3.19,1.23,121.1,93.1,1.64,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7168,김병준,99742,1977-04-01,2000.0,한화,23,22,0,0,2,...,25,1,3,5.67,6.82,1.79,83.8,69.8,-0.00,NaN
7169,김병준,99742,1977-04-01,2001.0,한화,24,19,0,0,0,...,21,1,1,2.93,6.08,1.83,166.5,80.4,0.51,NaN
7170,김병준,99742,1977-04-01,2002.0,한화,25,15,0,0,2,...,17,0,0,6.15,3.72,1.56,70.7,116.8,0.31,NaN
7171,전하진,99767,1979-02-11,2000.0,한화,21,7,0,0,0,...,8,0,0,5.40,5.56,1.63,88.0,85.5,0.12,NaN


In [33]:
pitcher_data_df_preprocessed['연도'] = pitcher_data_df_preprocessed['연도'].astype('int')

### Change data type to string

In [34]:
for col in pitcher_data_df_preprocessed.columns:
    pitcher_data_df_preprocessed[col] = pitcher_data_df_preprocessed[col].astype('str')

In [35]:
pitcher_data_df_preprocessed.dtypes

name        object
id          object
birthday    object
연도          object
팀           object
나이          object
출장          object
완투          object
완봉          object
선발          object
승           object
패           object
세           object
홀드          object
이닝          object
실점          object
자책          object
타자          object
안타          object
2타          object
3타          object
홈런          object
볼넷          object
고4          object
사구          object
삼진          object
보크          object
폭투          object
ERA         object
FIP         object
WHIP        object
ERA+        object
FIP+        object
WAR         object
WPA         object
dtype: object

In [36]:
pitcher_data_df_preprocessed

,name,id,birthday,연도,팀,나이,출장,완투,완봉,선발,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
0,황태환,10082,1952-08-06,1982,OB,30,27,2,0,6,...,42,0,4,3.86,4.85,1.31,100.9,80.2,1.08,nan
1,황태환,10082,1952-08-06,1983,OB,31,39,0,0,2,...,60,0,0,2.65,3.5,1.38,123.6,93.5,2.02,nan
2,황태환,10082,1952-08-06,1984,OB,32,17,0,0,3,...,19,0,0,2.48,3.44,1.28,135.7,97.6,0.92,nan
3,황태환,10082,1952-08-06,1985,OB,33,35,1,1,6,...,50,0,2,3.53,3.23,1.23,101.1,110.5,2.1,nan
4,황태환,10082,1952-08-06,1986,OB,34,28,0,0,1,...,36,0,1,2.45,3.19,1.23,121.1,93.1,1.64,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7168,김병준,99742,1977-04-01,2000,한화,23,22,0,0,2,...,25,1,3,5.67,6.82,1.79,83.8,69.8,-0.0,nan
7169,김병준,99742,1977-04-01,2001,한화,24,19,0,0,0,...,21,1,1,2.93,6.08,1.83,166.5,80.4,0.51,nan
7170,김병준,99742,1977-04-01,2002,한화,25,15,0,0,2,...,17,0,0,6.15,3.72,1.56,70.7,116.8,0.31,nan
7171,전하진,99767,1979-02-11,2000,한화,21,7,0,0,0,...,8,0,0,5.4,5.56,1.63,88.0,85.5,0.12,nan


In [37]:
pitcher_data_df_preprocessed.to_csv("./datasets/pitcher_detailed_stats.csv", index=False,
                                    encoding = "utf-8")

### 팀명 대문자로 바꿔주기

In [2]:
pitcher_data_df_preprocessed = pd.read_csv("./datasets/pitcher_detailed_stats.csv")

In [5]:
pitcher_data_df_preprocessed['팀'] = pitcher_data_df_preprocessed['팀'].str.upper()

In [7]:
pitcher_data_df_preprocessed.to_csv("./datasets/pitcher_detailed_stats.csv", index=False,
                                    encoding = "utf-8")

## Hitter data

In [38]:
def extract_hitter_data(url, name, id):
    response = requests.get(url, verify = False)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.select("body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(4) > div > div > table")
    if len(table) == 0: # can't find the player data
        return pd.DataFrame()
    else:
        colhead_rows = soup.find_all('tr', class_='colhead_stz0')
        for tr in colhead_rows[2:]:
            tr.decompose()
    
        hitter_df = pd.read_html(io.StringIO(str(table)))[0][:-1] # Remove "통산" row
        if hitter_df.empty:
            return hitter_df # no-data
                
        hitter_df.columns = hitter_df.columns.droplevel(0)
                
        # insert player name, id, birthday
        extracted_hitter_info = hitter_simple[(hitter_simple['name'] == name) & (hitter_simple['id'] == id)]
        extracted_hitter_info = extracted_hitter_info[extracted_hitter_info.columns[::-1]]
        
        
        for column in extracted_hitter_info:
            hitter_df.insert(0, column, extracted_hitter_info[column].values[0])
    
        return hitter_df


In [39]:
hitter_columns = ['name', 'id', 'birthday', '연도', '팀', '나이', 'P', 'G', '타석', '타수',
              '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷',
              '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS',
              'wOBA', 'wRC+', 'WAR*', 'WPA']

### Extract

In [40]:
hitter_url_head = 'http://www.statiz.co.kr/player.php?opt=1&name='
hitter_url_middle = '&birth='
hitter_url_tail = '&re=0'

In [45]:
hitter_data_list = list()
for i in tqdm(range(len(hitter_simple))):
    hitter_name = hitter_simple.iloc[i]['name']
    hitter_birthday = hitter_simple.iloc[i]['birthday']
    hitter_id = hitter_simple.iloc[i]['id']
    hitter_url = hitter_url_head+hitter_name+hitter_url_middle+hitter_birthday+hitter_url_tail
    hitter_data = extract_hitter_data(hitter_url, hitter_name, hitter_id)
    hitter_data_list = hitter_data_list + hitter_data.values.tolist()
    

100%|██████████| 2449/2449 [07:54<00:00,  5.16it/s]


In [46]:
hitter_data_df = pd.DataFrame(hitter_data_list, columns = hitter_columns)

### NaN처리
한 시즌 중에 선수가 팀을 옮긴 경우, 총 세가지의 column이 생기게 됨.  
ex.  
|      |   name |    id |   birthday |   연도 |      팀 | 나이 | 출장 |
|------|-------:|------:|-----------:|-------:|--------:|-----:|-----:|
| 7064 | 송신영 | 99314 | 1977-03-01 | 2011.0 | 넥센,LG |   34 |   62 |
| 7065 | 송신영 | 99314 | 1977-03-01 |    NaN |    넥센 |   34 |   43 |
| 7066 | 송신영 | 99314 | 1977-03-01 |    NaN |      LG |   34 |   19 |  

이렇게 연도가 NaN으로 표시되는 문제 해결

In [48]:
hitter_data_df_preprocessed = hitter_data_df.copy(deep=True)
hitter_data_df_preprocessed['연도'] = hitter_data_df['연도'].ffill()

In [49]:
hitter_data_df_preprocessed

,name,id,birthday,연도,팀,나이,P,G,타석,타수,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,차영화,10005,1957-06-27,1982.0,해태,25,SS,76,323,282,...,5,2,0.259,0.337,0.309,0.645,0.305,82.5,1.63,NaN
1,차영화,10005,1957-06-27,1983.0,해태,26,SS,82,252,229,...,5,3,0.266,0.308,0.323,0.631,0.291,81.5,1.01,NaN
2,차영화,10005,1957-06-27,1984.0,해태,27,SS,69,185,162,...,6,1,0.204,0.274,0.235,0.508,0.241,43.4,-0.02,NaN
3,차영화,10005,1957-06-27,1985.0,해태,28,SS,106,266,238,...,15,0,0.240,0.276,0.269,0.545,0.246,43.5,-0.00,NaN
4,차영화,10005,1957-06-27,1986.0,해태,29,SS,85,192,176,...,6,0,0.199,0.242,0.222,0.464,0.220,30.8,-0.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10561,이진영,99810,1980-06-15,2014.0,LG,34,RF,119,468,412,...,2,9,0.325,0.384,0.425,0.809,0.364,108.5,2.00,1.96
10562,이진영,99810,1980-06-15,2015.0,LG,35,RF,103,350,301,...,2,2,0.256,0.351,0.369,0.719,0.328,91.3,0.23,0.22
10563,이진영,99810,1980-06-15,2016.0,kt,36,DH,115,424,371,...,0,5,0.332,0.403,0.480,0.883,0.400,125.2,1.66,1.63
10564,이진영,99810,1980-06-15,2017.0,kt,37,DH,102,284,262,...,4,0,0.290,0.336,0.420,0.756,0.332,87.0,0.05,0.83


In [50]:
hitter_data_df_preprocessed['연도'] = hitter_data_df_preprocessed['연도'].astype('int')

In [51]:
hitter_data_df_preprocessed['나이'] = hitter_data_df_preprocessed['나이'].astype('int')

In [52]:
hitter_data_df_preprocessed.dtypes

name         object
id            int64
birthday     object
연도            int64
팀            object
나이            int64
P            object
G             int64
타석            int64
타수            int64
득점            int64
안타            int64
2타            int64
3타            int64
홈런            int64
루타            int64
타점            int64
도루            int64
도실            int64
볼넷            int64
사구            int64
고4            int64
삼진            int64
병살            int64
희타            int64
희비            int64
타율          float64
출루          float64
장타          float64
OPS         float64
wOBA        float64
wRC+        float64
WAR*        float64
WPA         float64
dtype: object

### Change data type to string

In [53]:
for col in hitter_data_df_preprocessed.columns:
    hitter_data_df_preprocessed[col] = hitter_data_df_preprocessed[col].astype('str')

In [54]:
hitter_data_df_preprocessed.dtypes

name        object
id          object
birthday    object
연도          object
팀           object
나이          object
P           object
G           object
타석          object
타수          object
득점          object
안타          object
2타          object
3타          object
홈런          object
루타          object
타점          object
도루          object
도실          object
볼넷          object
사구          object
고4          object
삼진          object
병살          object
희타          object
희비          object
타율          object
출루          object
장타          object
OPS         object
wOBA        object
wRC+        object
WAR*        object
WPA         object
dtype: object

In [55]:
hitter_data_df_preprocessed

,name,id,birthday,연도,팀,나이,P,G,타석,타수,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,차영화,10005,1957-06-27,1982,해태,25,SS,76,323,282,...,5,2,0.259,0.337,0.309,0.645,0.305,82.5,1.63,nan
1,차영화,10005,1957-06-27,1983,해태,26,SS,82,252,229,...,5,3,0.266,0.308,0.323,0.631,0.291,81.5,1.01,nan
2,차영화,10005,1957-06-27,1984,해태,27,SS,69,185,162,...,6,1,0.204,0.274,0.235,0.508,0.241,43.4,-0.02,nan
3,차영화,10005,1957-06-27,1985,해태,28,SS,106,266,238,...,15,0,0.24,0.276,0.269,0.545,0.246,43.5,-0.0,nan
4,차영화,10005,1957-06-27,1986,해태,29,SS,85,192,176,...,6,0,0.199,0.242,0.222,0.464,0.22,30.8,-0.46,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10561,이진영,99810,1980-06-15,2014,LG,34,RF,119,468,412,...,2,9,0.325,0.384,0.425,0.809,0.364,108.5,2.0,1.96
10562,이진영,99810,1980-06-15,2015,LG,35,RF,103,350,301,...,2,2,0.256,0.351,0.369,0.719,0.328,91.3,0.23,0.22
10563,이진영,99810,1980-06-15,2016,kt,36,DH,115,424,371,...,0,5,0.332,0.403,0.48,0.883,0.4,125.2,1.66,1.63
10564,이진영,99810,1980-06-15,2017,kt,37,DH,102,284,262,...,4,0,0.29,0.336,0.42,0.756,0.332,87.0,0.05,0.83


In [56]:
hitter_data_df_preprocessed.to_csv("./datasets/hitter_detailed_stats.csv", index=False,
                                    encoding = "utf-8")

### 팀명 대문자로 바꿔주기

In [8]:
hitter_data_df_preprocessed = pd.read_csv("./datasets/hitter_detailed_stats.csv")

In [9]:
hitter_data_df_preprocessed['팀'] = hitter_data_df_preprocessed['팀'].str.upper()

In [10]:
hitter_data_df_preprocessed.to_csv("./datasets/hitter_detailed_stats.csv", index=False,
                                    encoding = "utf-8")